In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

/var/folders/81/_1yshp9s2sxb1nghxqpzk7b40000gn/T/ipykernel_6185/3377578104.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [9]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

display(df_train.sample(5)) # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
11544,0.5,2100 Alaskan Way /Seattle/,ID,NaN,16101,NaN,SMB,NaN,NaN,NaN,...,LGEUS,less than 3 months,0,0,NaN,NaN,NaN,NaN,194,False
5557,1.0,/DUBAI/U.A.E,ID,NaN,40824,NaN,SMB,NaN,NaN,NaN,...,LGEGF,less than 3 months,0,0,NaN,NaN,NaN,NaN,229,False
697,1.0,//ARGENTINA,ID,0.074949,33673,NaN,SMB,57.0,1.0,NaN,...,LGEAR,3 months ~ 6 months,0,0,0.003079,0.064566,corporate / office,NaN,237,False
27050,1.0,/Maldives,ID,NaN,39687,NaN,SMB,NaN,1.0,NaN,...,LGESL,less than 3 months,0,1,0.000717,0.071345,hotel & accommodation,NaN,926,False
53541,1.0,/Pasto/Colombia,ID,0.032787,43789,End-Customer,SMB,NaN,NaN,NaN,...,LGECB,less than 3 months,1,0,0.000572,0.048630,education,Higher Education (College & University),233,False


In [5]:
"""
*/*/country -> country
"""

def column_short(df : pd.DataFrame, column : str):
    short_column = column+'_short'
    df[short_column] = "others"
    df[short_column][df[column].notna()] = df.copy()[df[column].notna()][column].apply(lambda x : x.split('/')[-1].split(',')[-1].strip().lower())
    df[short_column].replace("", "others", inplace=True)
    # write aaa@aa.com -> others regex : r'@.*'
    df[short_column].replace(to_replace=r'^[a-zA-Z0-9][a-zA-Z0-9._]+[@][a-zA-Z][A-Za-z.]+[.]\w{2,}', value="others", inplace=True, regex=True)


def column_replace(df : pd.DataFrame, column : str, replace_dict : dict):
    df[column].replace(r"*[]", inplace=True, regex=True)

In [ ]:
def target_ratio(column : str, nums : int):
    targets_dict = df_train[df_train[column] == nums]['is_converted'].value_counts(ascending=False).to_dict()
    if len(targets_dict) == 1:
        if True in targets_dict.keys():
            F_target = 0
            T_target = targets_dict[True]
        elif False in targets_dict.keys():
            T_target = 0
            F_target = targets_dict[False]
    else:
        F_target = targets_dict[False]
        T_target = targets_dict[True]
    if F_target == 0:
        print("{0} value of {1}".format(nums, column)+"'s target ratio : {0} / {1}, only True".format(T_target, F_target), "\n")
    else:
        print("{0} value of {1}".format(nums, column)+"'s target ratio : {0} / {1} = {2:.2f} %".format(T_target, F_target, T_target / F_target * 100), "\n")

### Column : bant_submit

In [ ]:
"""
0, 0.25, 0.5, 0.75, 1 의 class가 존재

class 별로 target 비율에 대해서 고려하기

1 : 9.23%
0.75 : 10.57%
0.5 : 5.76%
0.25 : 10.98%
0 : 19.03%

0일때, 0.5일때 꽤 유의미한 차이가 있고, 1, 0.75, 0.25는 비슷한 비율을 가지고 있다.
"""
columns = 'bant_submit'
for nums in df_train[columns].unique():
    target_ratio(columns, nums)

### Column : customer_country

In [ ]:
def show_customer_country_from_short(country_short : str):
    print(df_train[df_train['customer_country_short'] == country_short].customer_country.value_counts())

### Column : business_unit

In [ ]:
columns = 'business_unit'
for nums in df_train[columns].unique():
    target_ratio(columns, nums)

### Column : com_reg_ver_win_rate

In [ ]:
columns = 'com_reg_ver_win_rate'
temp_dict = df_train[columns].value_counts().to_dict()
temp_dict = {k: v for k, v in sorted(temp_dict.items(), key=lambda x : x[0], reverse=True)}

for k, v in temp_dict.items():
    target_ratio(columns, k)

### Column : customer_idx

In [ ]:
columns = 'customer_idx'
df_train[columns].value_counts()

In [ ]:
columns = 'customer_idx'
temp_dict = df_train[columns].value_counts().to_dict()
temp_dict = {k: v for k, v in sorted(temp_dict.items(), key=lambda x:-x[1])}

for_plot = []
def temp_target_ratio(column : str, nums : int, value : int):
    targets_dict = df_train[df_train[column] == nums]['is_converted'].value_counts(ascending=False).to_dict()
    if len(targets_dict) == 1:
        if True in targets_dict.keys():
            # F_target = 0
            T_target = targets_dict[True]
        elif False in targets_dict.keys():
            T_target = 0
            # F_target = targets_dict[False]
    else:
        T_target = targets_dict[True]
    # print("{0} value of {1}".format(nums, column)+"'s target ratio : {0} / {1} = {2:.2f} %".format(T_target, value, T_target / value * 100), "\n")
    for_plot.append(round(T_target / value * 100, 2))

for k, v in temp_dict.items():
    temp_target_ratio(columns, k, v)

In [ ]:
sns.kdeplot(for_plot)
plt.show()

from collections import Counter
print(len(for_plot))
print(Counter(for_plot))


### Column : customer_type

### Column : Enterpise

### Column : historical_existing_cnt

### Column : id_strategic_ver

### Column : it_strategic_ver

### Column : idit_strategic_ver

### Column : customer_job

### Column : lead_desc_length

### Column : inquiry_type

### Column : product_category

### Column : product_subcategory

### Column : product_modelname

### Column : customer_country.1

### Column : customer_position

### Column : reponse_corporate

### Column : expected_timeline

### Column : vers_cus

### Column : vers_pro

### Column : ver_win_rate_x

### Column : ver_win_ratio_per_bu

### Column : business_area

### Column : business_subarea

### Column : lead_owner

### Column : is_converted